In [1]:
import numpy as np


In [2]:
class NeuralNetwork:
    
    #initialise neural network
    def __init__(self, givenINodes, givenHNodes, givenONodes, givenLearningRate):
        #setup number of nodes in each layer
        self.inputNodes = givenINodes
        self.hiddenNodes = givenHNodes
        self.outputNodes = givenONodes
        
        #setup weights with initial values
        self.weightsInputHidden = np.random.normal(0.0, pow(self.inputNodes, -0.5), (self.hiddenNodes, self.inputNodes))
        self.weightsHiddenOutput = np.random.normal(0.0, pow(self.hiddenNodes, -0.5), (self.outputNodes, self.hiddenNodes))
        
        #setup learning rate
        self.learningRate = givenLearningRate
        
        #setup activation function
        self.activationFunc = lambda x: self.sigmoid(x)
            
        pass
            
    def sigmoid(self, Z):
        A = 1 / (1 + np.exp(-Z))
        return A
    
    #train the neural network
    def train(self, givenInputs, givenTargets):
        #inputs and target lists turned into 2D arrays
        inputs = np.array(givenInputs, ndmin=2).T
        targets = np.array(givenTargets, ndmin=2).T
        
        #signals into hidden layer calculated
        hiddenInputs = np.dot(self.weightsInputHidden, inputs)
        #signals coming from hidden layer calculated
        hiddenOutputs = self.activationFunc(hiddenInputs)
        
        #signals into output layer calculated
        finalInputs = np.dot(self.weightsHiddenOutput, hiddenOutputs)
        #signals from output layer calculated
        finalOutputs = self.activationFunc(finalInputs)
        
        #error backpropagation:
        #errors from output layer
        outputErrors = targets - finalOutputs
        #errors from hidden layer
        hiddenErrors = np.dot(self.weightsHiddenOutput.T, outputErrors)
        
        #weight links updated between all layers
        self.weightsHiddenOutput += self.learningRate * np.dot((outputErrors * finalOutputs * (1.0 - finalOutputs)), np.transpose(hiddenOutputs))
        self.weightsInputHidden += self.learningRate * np.dot((hiddenErrors * hiddenOutputs * (1.0 - hiddenOutputs)), np.transpose(inputs))
        
        pass
    
    #query the neural network
    def query(self, givenInputs):
        #inputs list converted into a 2D array
        inputs = np.array(givenInputs, ndmin=2).T
        
        #signals into hidden layer calculated
        hiddenInputs = np.dot(self.weightsInputHidden, inputs)
        #signals coming from hidden layer calculated
        hiddenOutputs = self.activationFunc(hiddenInputs)
        
        #signals into output layer calculated
        finalInputs = np.dot(self.weightsHiddenOutput, hiddenOutputs)
        #final signals from neural net calculated
        finalOutputs = self.activationFunc(finalInputs)
        
        return finalOutputs
    
        

In [3]:
#set number of nodes in each layer
inputNodes = 784
hiddenNodes = 200
outputNodes = 10

#set learning rate
learningRate = 0.1

#an instance of the neural network created
n = NeuralNetwork(inputNodes, hiddenNodes, outputNodes, learningRate)

In [4]:
#open training data file in READ mode
trainingDataFile = open("mnist_dataset/mnist_train.csv",'r')
#save all the lines from the file into a list
trainingDataList = trainingDataFile.readlines()
#close the file
trainingDataFile.close()

In [6]:
#trainPrep

#number of times training data used for training
epochs = 10

for e in range(epochs):
    #for every record in the training data
    for record in trainingDataList:
        #split each record by the commas
        allVals = record.split(',')
        #scale and shift the inputs
        inputs = (np.asfarray(allVals[1:]) / 255.0 * 0.99) + 0.01
        #print ("Training network", e)
        #target output values created
        targets = np.zeros(outputNodes) + 0.01
        targets[int(allVals[0])] = 0.99
        n.train(inputs, targets)
        
        pass
    pass

In [7]:
#open test data file in READ mode
testDataFile = open("mnist_dataset/mnist_test_10.csv",'r')
#save all the lines from the file into a list
testDataList = testDataFile.readlines()
#close the file
testDataFile.close()

In [8]:

# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in testDataList:
    # split the record by the ',' commas
    allVals = record.split(',')
    # correct answer is first value
    correct_label = int(allVals[0])
    # scale and shift the inputs
    inputs = (np.asfarray(allVals[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [10]:
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
scorecard_percent = (scorecard_array.sum() / scorecard_array.size) * 100
print ("Calculated Network Performance = ", scorecard_array.sum() / scorecard_array.size)
print ("Network Accuracy = ", scorecard_percent, "%")


Calculated Network Performance =  1.0
Network Accuracy =  100.0 %
